# 1. Data Information

In [1]:
import pandas as pd
import numpy as np

# Dataset
file_path = 'data/IncidentTableData.csv'
df = pd.read_csv(file_path)

# shape
print('Shape of the dataset:', df.shape)

# column names
print('\nColumn names:', df.columns.tolist())

# data types
print('\nData types:')
print(df.dtypes)


Shape of the dataset: (102, 30)

Column names: ['Ticket Class', 'Ticket Priority', 'Ticket Number', 'Ticket Status', 'Ticket Status Original', 'Opened Date', 'Hostname', 'OS Type', 'Ticket Summary', 'Queue ID', 'Ticket Resolved Date', 'Resolution Code', 'Resolution Text', 'Ticket Closed Date', 'Call code', 'Reported By', 'Executed Automata', 'Recommended Automata', 'Actionable', 'Assignment Queue', 'Autogenerated', 'Automation Engine', 'Business Application', 'Closure Code', 'Sub Category', 'Target Finish Date', 'Time to resolve (min)', 'Category', 'Alert Key', 'Comments & Work Notes']

Data types:
Ticket Class               object
Ticket Priority             int64
Ticket Number              object
Ticket Status              object
Ticket Status Original     object
Opened Date                object
Hostname                   object
OS Type                    object
Ticket Summary             object
Queue ID                   object
Ticket Resolved Date       object
Resolution Code     

In [2]:
# missing values count
print('\nMissing values per column:')
print(df.isnull().sum())


Missing values per column:
Ticket Class                0
Ticket Priority             0
Ticket Number               0
Ticket Status               0
Ticket Status Original      0
Opened Date                 0
Hostname                    4
OS Type                     4
Ticket Summary              0
Queue ID                    0
Ticket Resolved Date      101
Resolution Code             0
Resolution Text            24
Ticket Closed Date        102
Call code                   0
Reported By                 0
Executed Automata         102
Recommended Automata       65
Actionable                  0
Assignment Queue            0
Autogenerated               1
Automation Engine           3
Business Application       22
Closure Code              102
Sub Category              101
Target Finish Date        102
Time to resolve (min)     101
Category                    0
Alert Key                   6
Comments & Work Notes     102
dtype: int64


In [3]:
# basic stats
print('\nBasic statistics:')
print(df.describe(include='all'))


Basic statistics:
       Ticket Class  Ticket Priority Ticket Number Ticket Status  \
count           102       102.000000           102           102   
unique            1              NaN           102             3   
top        INCIDENT              NaN    IN89157912      SLA HOLD   
freq            102              NaN             1            61   
mean            NaN         1.794118           NaN           NaN   
std             NaN         0.406342           NaN           NaN   
min             NaN         1.000000           NaN           NaN   
25%             NaN         2.000000           NaN           NaN   
50%             NaN         2.000000           NaN           NaN   
75%             NaN         2.000000           NaN           NaN   
max             NaN         2.000000           NaN           NaN   

       Ticket Status Original          Opened Date       Hostname OS Type  \
count                     102                  102             98      98   
unique    

# 2. Data Cleaning


In [4]:
# column name standardization : remove spaces and convert to lowercase
df.columns = [col.strip().replace(' ', '_').lower() for col in df.columns]

In [5]:
# remove duplicate rows
df.drop_duplicates(inplace=True)


In [6]:
# Correct data types for date columns
date_cols = ['opened_date', 'ticket_resolved_date', 'ticket_closed_date', 'target_finish_date']
for col in date_cols:
    if col in df.columns:
        df[col] = pd.to_datetime(df[col], errors='coerce')

print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102 entries, 0 to 101
Data columns (total 30 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   ticket_class            102 non-null    object        
 1   ticket_priority         102 non-null    int64         
 2   ticket_number           102 non-null    object        
 3   ticket_status           102 non-null    object        
 4   ticket_status_original  102 non-null    object        
 5   opened_date             102 non-null    datetime64[ns]
 6   hostname                98 non-null     object        
 7   os_type                 98 non-null     object        
 8   ticket_summary          102 non-null    object        
 9   queue_id                102 non-null    object        
 10  ticket_resolved_date    1 non-null      datetime64[ns]
 11  resolution_code         102 non-null    object        
 12  resolution_text         78 non-null     object    

# 3. Data Enhancement

In [7]:
# Extract date features from opened_date
if 'opened_date' in df.columns:
    df['opened_year'] = df['opened_date'].dt.year
    df['opened_month'] = df['opened_date'].dt.month
    df['opened_day'] = df['opened_date'].dt.day
    df['opened_hour'] = df['opened_date'].dt.hour


In [8]:
# Convert all object columns to string
df = df.astype({col: 'string' for col in df.select_dtypes(include='object').columns})

In [9]:
# Normalize categorical values: convert to lowercase
categorical_cols = ['ticket_class', 'ticket_priority', 'ticket_status', 'hostname', 'os_type', 'business_application']
for col in categorical_cols:
    if col in df.columns:
        df[col] = df[col].astype('string')
        df[col] = df[col].str.lower()

print(df.head())

  ticket_class ticket_priority ticket_number ticket_status  \
0     incident               2    IN89157912   in progress   
1     incident               1    IN88258016   in progress   
2     incident               2    IN71887046        queued   
3     incident               2    IN71887860        queued   
4     incident               2    IN71841189      sla hold   

  ticket_status_original         opened_date       hostname os_type  \
0            IN PROGRESS 2025-11-16 04:30:00   fgceogqto405  vmware   
1            IN PROGRESS 2025-11-16 01:00:00  fgkdoqtqohif5   linux   
2                 QUEUED 2025-11-15 19:06:00       ngfed139   linux   
3                 QUEUED 2025-11-15 18:06:00       ngfed139   linux   
4                SLAHOLD 2025-11-15 14:03:00           <NA>    <NA>   

                                      ticket_summary  \
0  fgceogqto405 08-CPU load is high on processor ...   
1  acme /dev/PSYIXPRD::/PRD: : File system usage ...   
2  edd#ledcb795#SAP: PA1-sappa1a

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102 entries, 0 to 101
Data columns (total 34 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   ticket_class            102 non-null    string        
 1   ticket_priority         102 non-null    string        
 2   ticket_number           102 non-null    string        
 3   ticket_status           102 non-null    string        
 4   ticket_status_original  102 non-null    string        
 5   opened_date             102 non-null    datetime64[ns]
 6   hostname                98 non-null     string        
 7   os_type                 98 non-null     string        
 8   ticket_summary          102 non-null    string        
 9   queue_id                102 non-null    string        
 10  ticket_resolved_date    1 non-null      datetime64[ns]
 11  resolution_code         102 non-null    string        
 12  resolution_text         78 non-null     string    

# 4. Derived Fields

In [11]:
# Resolution Time (hours)
if 'ticket_resolved_date' in df.columns and 'opened_date' in df.columns:
    df['resolution_time_hours'] = (df['ticket_resolved_date'] - df['opened_date']).dt.total_seconds() / 3600


In [12]:
# Ticket Age (days)
current_time = pd.Timestamp.now()
if 'opened_date' in df.columns:
    df['ticket_age_days'] = (current_time - df['opened_date']).dt.days


In [13]:
# Is Automated ('executed_automata' column)
if 'executed_automata' in df.columns:
    df['is_automated'] = df['executed_automata'].apply(lambda x: False if x == '' else True)


In [14]:
# Critical application flag (based on business_application column)
critical_apps = ['payment processing', 'internet banking', 'my web application']
if 'business_application' in df.columns:
    df['critical_application_flag'] = df['business_application'].apply(lambda x: True if isinstance(x, str) and x in critical_apps else False)


In [15]:
print(df[['resolution_time_hours', 'ticket_age_days', 'is_automated', 'critical_application_flag']].head())

   resolution_time_hours  ticket_age_days  is_automated  \
0                    NaN                0          True   
1               0.083333                0          True   
2                    NaN                0          True   
3                    NaN                0          True   
4                    NaN                1          True   

   critical_application_flag  
0                       True  
1                       True  
2                      False  
3                      False  
4                      False  


# 5. Cleaned Data Export

In [ ]:
df.to_csv("data/_cleaned.csv", index=False)